In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- 1. Gelişmiş Ayarlar ve Sabitler ---
START_DATE = datetime(2025, 1, 1)
days_to_generate = 90  # 3 Aylık veri
END_DATE = START_DATE + timedelta(days=days_to_generate)
EMPLOYEE_COUNT = 100

# Olasılık Oranları (0.0 - 1.0 arası)
RATES = {
    'ERROR': 0.12,      # %12 Hatalı kayıt (Geç kalma / Erken çıkma)
    'LEAVE': 0.05,      # %5 İzinli olma ihtimali
    'ABSENCE': 0.02,    # %2 Mazeretsiz devamsızlık
    'OVERTIME': 0.20,   # %20 Fazla mesai ihtimali (Özellikle Üretim için)
    'HYBRID_WFH': 0.40  # Hibrit personelin evden çalışma ihtimali (o gün için)
}

DEPARTMENTS = ['Üretim', 'AR-GE', 'Muhasebe', 'İnsan Kaynakları', 'Lojistik']

# Vardiya Saatleri (Başlangıç saati, Çalışma süresi saati)
SHIFT_CONFIG = {
    'Ofis': {'start': 9, 'hours': 9},    # 09:00 - 18:00
    'Üretim': {'start': 8, 'hours': 9},  # 08:00 - 17:00
    'Hibrit': {'start': 10, 'hours': 9}  # 10:00 - 19:00 (Esnek)
}

# --- 2. Personel Listesi Oluşturma ---
employees = []
for i in range(EMPLOYEE_COUNT):
    dept = random.choice(DEPARTMENTS)
    # Üretim ve Lojistik genellikle hibrit olamaz varsayımı
    is_hybrid = False if dept in ['Üretim', 'Lojistik'] else random.choice([True, False])

    employees.append({
        'Employee_ID': 1000 + i,
        'Name': f"Personel_{1000+i}", # İsim simülasyonu
        'Department': dept,
        'Is_Hybrid': is_hybrid
    })

employees_df = pd.DataFrame(employees)

# --- 3. Günlük Veri Döngüsü ---
daily_records = []
current_date = START_DATE

while current_date <= END_DATE:
    day_str = current_date.strftime('%Y-%m-%d')
    is_weekend = current_date.weekday() >= 5 # 5: Cumartesi, 6: Pazar
    week_num = current_date.isocalendar()[1] # Hafta numarası

    for _, emp in employees_df.iterrows():
        emp_id = emp['Employee_ID']
        dept = emp['Department']

        # --- A. Hafta Sonu Kontrolü ---
        # Sadece Üretim ve Lojistik bazen hafta sonu çalışır
        if is_weekend:
            if dept not in ['Üretim', 'Lojistik'] or random.random() > 0.3:
                continue # Kayıt atma (Hafta tatili)

        # --- B. Statü Belirleme (İzin/Devamsızlık/Çalışma) ---
        status = 'Çalışıyor'
        error_type = 'Yok'
        location = 'Ofis'

        rand_val = random.random()

        if rand_val < RATES['ABSENCE']:
            status = 'Devamsızlık (M)'
            error_type = 'Mazeretsiz Gelmeme'
        elif rand_val < RATES['ABSENCE'] + RATES['LEAVE']:
            status = random.choice(['Yıllık İzin', 'Raporlu', 'İdari İzin'])
            error_type = 'İzinli'

        # Eğer çalışmıyorsa saatleri boş geç
        if status != 'Çalışıyor':
            daily_records.append({
                'Date': current_date,
                'Day_Str': day_str,
                'Week_Number': week_num,
                'Employee_ID': emp_id,
                'Department': dept,
                'Work_Status': status,
                'Location': 'Ev/Dışarı',
                'Scheduled_Start': None,
                'Actual_Start': None,
                'Actual_End': None,
                'Overtime_Hours': 0.0,
                'Error_Type': error_type
            })
            continue

        # --- C. Çalışma Saatleri Simülasyonu ---

        # Vardiya Tipi Belirleme
        if emp['Is_Hybrid']:
            # Hibrit çalışan o gün evde mi ofiste mi?
            if random.random() < RATES['HYBRID_WFH']:
                location = 'Ev (Remote)'
                shift_conf = SHIFT_CONFIG['Hibrit']
            else:
                location = 'Ofis'
                shift_conf = SHIFT_CONFIG['Ofis'] # Ofise gelince standart saate uyuyor varsayalım
        elif dept == 'Üretim':
            shift_conf = SHIFT_CONFIG['Üretim']
        else:
            shift_conf = SHIFT_CONFIG['Ofis']

        # Planlanan Saatler
        sched_start = current_date.replace(hour=shift_conf['start'], minute=0, second=0)
        sched_end = sched_start + timedelta(hours=shift_conf['hours'])

        # Gerçekleşen Saatler (Sapmalar)
        actual_start = sched_start
        actual_end = sched_end

        # Hata/Anomali Enjeksiyonu (Geç kalma / Erken çıkma)
        if random.random() < RATES['ERROR']:
            if random.choice([True, False]):
                # Geç Giriş (15 - 120 dk arası)
                delay = random.randint(15, 120)
                actual_start = sched_start + timedelta(minutes=delay)
                error_type = 'Geç Giriş'
            else:
                # Erken Çıkış (15 - 60 dk arası)
                early = random.randint(15, 60)
                actual_end = sched_end - timedelta(minutes=early)
                error_type = 'Erken Çıkış'

        # Fazla Mesai Enjeksiyonu
        ot_hours = 0.0
        if status == 'Çalışıyor' and error_type == 'Yok' and random.random() < RATES['OVERTIME']:
            # Genelde üretim veya yoğun dönemlerde
            if dept in ['Üretim', 'Lojistik', 'AR-GE']:
                ot_minutes = random.randint(30, 240) # 30dk ile 4 saat arası
                actual_end = actual_end + timedelta(minutes=ot_minutes)
                ot_hours = round(ot_minutes / 60, 2)
                error_type = 'Fazla Mesai'

        daily_records.append({
            'Date': current_date,
            'Day_Str': day_str,
            'Week_Number': week_num,
            'Employee_ID': emp_id,
            'Department': dept,
            'Work_Status': status,
            'Location': location,
            'Scheduled_Start': sched_start.strftime('%H:%M'),
            'Actual_Start': actual_start.strftime('%H:%M'),
            'Actual_End': actual_end.strftime('%H:%M'),
            'Overtime_Hours': ot_hours,
            'Error_Type': error_type
        })

    current_date += timedelta(days=1)

# DataFrame'e dönüştür
df = pd.DataFrame(daily_records)

# --- 4. Haftalık İletişim Simülasyonu ---
# Senaryo: Her hafta bitiminde, o hafta içinde "Fazla Mesai" yapan veya "Devamsızlık" yapanlara mail gider.

def calculate_weekly_comm(group):
    # O haftaki toplam mesai
    total_ot = group['Overtime_Hours'].sum()
    # O haftaki devamsızlık sayısı
    absences = group[group['Work_Status'] == 'Devamsızlık (M)'].shape[0]

    msg = "Standart Haftalık Rapor"
    if total_ot > 0:
        msg = f"Tebrikler, bu hafta toplam {total_ot} saat fazla mesai onaylandı."
    if absences > 0:
        msg = "DİKKAT: Bu hafta mazeretsiz devamsızlık kaydınız bulunmaktadır."
    if total_ot > 0 and absences > 0:
        msg = f"Mesai: {total_ot} saat. UYARI: Devamsızlık kaydı mevcut."

    group['Weekly_Notification_Msg'] = msg
    return group

# Veriyi Gruplayıp Mesajı Dağıtma
df = df.groupby(['Employee_ID', 'Week_Number'], group_keys=False).apply(calculate_weekly_comm)

# --- 5. NLP İçin Talep Metni Oluşturma (HR_Ticket_Text) ---
# Veri setini zenginleştirmek için her satıra bir "İK Talebi / Logu" metni ekleyelim.

def create_nlp_text(row):
    status = row['Work_Status']
    err = row['Error_Type']
    ot = row['Overtime_Hours']

    if status == 'Devamsızlık (M)':
        return "Personel işe gelmedi, mazeret bildirmedi. Tutanak oluşturulmalı."
    elif status == 'Raporlu':
        return "Personel sağlık raporunu WhatsApp üzerinden iletti, sisteme işlenecek."
    elif err == 'Geç Giriş':
        return f"Personel mesaiye {row['Actual_Start']} saatinde giriş yaptı, geç kalma nedeni sorulmalı."
    elif err == 'Erken Çıkış':
        return "Mesai bitiminden önce çıkış yapıldı, yönetici onayı kontrol ediliyor."
    elif ot > 0:
        return f"Haftalık üretim hedefleri için {ot} saat fazla çalışma yapıldı, ödeme listesine ekle."
    elif row['Location'] == 'Ev (Remote)':
        return "Personel bugün uzaktan çalışıyor, VPN logları kontrol edildi."
    else:
        return "Rutin çalışma kaydı, sorun yok."

df['HR_Log_Text'] = df.apply(create_nlp_text, axis=1)

# --- 6. Sonuçları Gösterme ve Kaydetme ---
print(f"✅ Veri seti oluşturuldu. Toplam Satır: {len(df)}")
print(f"📅 Tarih Aralığı: {START_DATE.strftime('%Y-%m-%d')} - {END_DATE.strftime('%Y-%m-%d')}")
print("\nÖrnek Veri (İlk 5 Satır):")
display_cols = ['Date', 'Employee_ID', 'Department', 'Work_Status', 'Actual_Start', 'Overtime_Hours', 'Error_Type', 'Weekly_Notification_Msg']
print(df[display_cols].head(10))

# CSV Olarak Kaydet (Colab'da sol menüdeki dosyalar kısmına gelir)
df.to_csv('hr_punch_data_with_scenarios.csv', index=False)
print("\n📁 Dosya kaydedildi: 'hr_punch_data_with_scenarios.csv'")

In [ ]:
toplam_benzersiz_id = df['Employee_ID'].nunique()
print(f"Toplam benzersiz çalışan ID sayısı: {toplam_benzersiz_id}")

In [ ]:
# HÜCRE 5: Sonuçların Görüntülenmesi ve Kayıt

print("\n--- Veri Seti Özeti ---")
print(f"Toplam Satır: {len(df)}")
print(f"📅 Tarih Aralığı: {df['Day_Str'].min()} - {df['Day_Str'].max()}")

print("\nEn Sık Görülen Anomali Tipleri:")
print(df['Error_Type'].value_counts())

print("\nÖrnek Veri (İlk 10 Satır):")
display_cols = ['Date', 'Employee_ID', 'Department', 'Work_Status', 'Location', 'Actual_Start', 'Overtime_Hours', 'Error_Type', 'Weekly_Notification_Msg']
print(df[display_cols].head(10))

# CSV Olarak Kaydetme
df.to_csv('hr_punch_data_with_scenarios.csv', index=False)
print("\n📁 Dosya kaydedildi: 'hr_punch_data_with_scenarios.csv'")

In [ ]:
# HÜCRE 1: Kütüphaneler, Sabitler ve Ayarlar

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- 1. Gelişmiş Ayarlar ve Sabitler ---
START_DATE = datetime(2025, 1, 1)
days_to_generate = 90  # 3 Aylık veri
END_DATE = START_DATE + timedelta(days=days_to_generate)
EMPLOYEE_COUNT = 3000

# Olasılık Oranları (0.0 - 1.0 arası)
RATES = {
    'ERROR': 0.12,      # %12 Hatalı kayıt (Geç kalma / Erken çıkma)
    'LEAVE': 0.05,      # %5 İzinli olma ihtimali
    'ABSENCE': 0.02,    # %2 Mazeretsiz devamsızlık
    'OVERTIME': 0.20,   # %20 Fazla mesai ihtimali (Özellikle Üretim için)
    'HYBRID_WFH': 0.40  # Hibrit personelin evden çalışma ihtimali (o gün için)
}

DEPARTMENTS = ['Üretim', 'AR-GE', 'Muhasebe', 'İnsan Kaynakları', 'Lojistik']

# Vardiya Saatleri (Başlangıç saati, Çalışma süresi saati)
SHIFT_CONFIG = {
    'Ofis': {'start': 9, 'hours': 9},    # 09:00 - 18:00
    'Üretim': {'start': 8, 'hours': 9},  # 08:00 - 17:00
    'Hibrit': {'start': 10, 'hours': 9}  # 10:00 - 19:00 (Esnek)
}

print("Ayarlar yüklendi. Veri üretimi hazır.")

In [ ]:
# HÜCRE 2: Personel Listesi Oluşturma

employees = []
for i in range(EMPLOYEE_COUNT):
    dept = random.choice(DEPARTMENTS)
    # Üretim ve Lojistik genellikle hibrit olamaz varsayımı
    is_hybrid = False if dept in ['Üretim', 'Lojistik'] else random.choice([True, False])

    employees.append({
        'Employee_ID': 1000 + i,
        'Name': f"Personel_{1000+i}", # İsim simülasyonu
        'Department': dept,
        'Is_Hybrid': is_hybrid
    })

employees_df = pd.DataFrame(employees)

print(f"✅ {EMPLOYEE_COUNT} adet personel kaydı oluşturuldu.")
print("\nPersonel Dağılımı:")
print(employees_df['Department'].value_counts())

In [ ]:
# HÜCRE 3: Günlük Zaman Yönetimi Kayıtlarını Oluşturma (Ana Döngü)

daily_records = []
current_date = START_DATE

print(f"\nGünlük kayıtlar üretiliyor ( {START_DATE.strftime('%Y-%m-%d')} - {END_DATE.strftime('%Y-%m-%d')} arası)...")

while current_date <= END_DATE:
    day_str = current_date.strftime('%Y-%m-%d')
    is_weekend = current_date.weekday() >= 5
    week_num = current_date.isocalendar()[1]

    for _, emp in employees_df.iterrows():
        emp_id = emp['Employee_ID']
        dept = emp['Department']

        # --- A. Hafta Sonu Kontrolü ---
        if is_weekend:
            if dept not in ['Üretim', 'Lojistik'] or random.random() > 0.3:
                continue

        # --- B. Statü Belirleme (İzin/Devamsızlık/Çalışma) ---
        status = 'Çalışıyor'
        error_type = 'Yok'
        location = 'Ofis'

        rand_val = random.random()

        if rand_val < RATES['ABSENCE']:
            status = 'Devamsızlık (M)'
            error_type = 'Mazeretsiz Gelmeme'
        elif rand_val < RATES['ABSENCE'] + RATES['LEAVE']:
            status = random.choice(['Yıllık İzin', 'Raporlu', 'İdari İzin'])
            error_type = 'İzinli'

        # Eğer çalışmıyorsa saatleri boş geç
        if status != 'Çalışıyor':
            daily_records.append({
                'Date': current_date,
                'Day_Str': day_str,
                'Week_Number': week_num,
                'Employee_ID': emp_id,
                'Department': dept,
                'Work_Status': status,
                'Location': 'İzinli/Devamsız',
                'Scheduled_Start': None,
                'Actual_Start': None,
                'Actual_End': None,
                'Overtime_Hours': 0.0,
                'Error_Type': error_type
            })
            continue

        # --- C. Çalışma Saatleri Simülasyonu ---

        # Vardiya Tipi Belirleme ve Lokasyon
        if emp['Is_Hybrid']:
            if random.random() < RATES['HYBRID_WFH']:
                location = 'Ev (Remote)'
                shift_conf = SHIFT_CONFIG['Hibrit']
            else:
                location = 'Ofis'
                shift_conf = SHIFT_CONFIG['Ofis']
        elif dept == 'Üretim':
            shift_conf = SHIFT_CONFIG['Üretim']
        else:
            shift_conf = SHIFT_CONFIG['Ofis']

        # Planlanan Saatler
        sched_start = current_date.replace(hour=shift_conf['start'], minute=0, second=0)
        sched_end = sched_start + timedelta(hours=shift_conf['hours'])

        # Gerçekleşen Saatler (Sapmalar)
        actual_start = sched_start
        actual_end = sched_end

        # Hata/Anomali Enjeksiyonu
        if random.random() < RATES['ERROR']:
            if random.choice([True, False]):
                delay = random.randint(15, 120)
                actual_start = sched_start + timedelta(minutes=delay)
                error_type = 'Geç Giriş'
            else:
                early = random.randint(15, 60)
                actual_end = sched_end - timedelta(minutes=early)
                error_type = 'Erken Çıkış'

        # Fazla Mesai Enjeksiyonu
        ot_hours = 0.0
        if status == 'Çalışıyor' and error_type == 'Yok' and random.random() < RATES['OVERTIME']:
            if dept in ['Üretim', 'Lojistik', 'AR-GE']:
                ot_minutes = random.randint(30, 240)
                actual_end = actual_end + timedelta(minutes=ot_minutes)
                ot_hours = round(ot_minutes / 60, 2)
                error_type = 'Fazla Mesai'

        daily_records.append({
            'Date': current_date,
            'Day_Str': day_str,
            'Week_Number': week_num,
            'Employee_ID': emp_id,
            'Department': dept,
            'Work_Status': status,
            'Location': location,
            'Scheduled_Start': sched_start.strftime('%H:%M'),
            'Actual_Start': actual_start.strftime('%H:%M'),
            'Actual_End': actual_end.strftime('%H:%M'),
            'Overtime_Hours': ot_hours,
            'Error_Type': error_type
        })

    current_date += timedelta(days=1)

# DataFrame'e dönüştür
df = pd.DataFrame(daily_records)
print(f"✅ Günlük kayıt döngüsü tamamlandı. Toplam Satır: {len(df)}")

In [ ]:
# HÜCRE 4: Haftalık İletişim ve NLP Metin Fonksiyonları

def calculate_weekly_comm(group):
    """Fazla mesai ve devamsızlığa göre haftalık iletişim mesajını oluşturur."""
    total_ot = group['Overtime_Hours'].sum()
    absences = group[group['Work_Status'] == 'Devamsızlık (M)'].shape[0]

    msg = "Standart Haftalık Rapor"
    if total_ot > 0:
        msg = f"Tebrikler, bu hafta toplam {total_ot:.2f} saat fazla mesai onaylandı."
    if absences > 0:
        msg = "DİKKAT: Bu hafta mazeretsiz devamsızlık kaydınız bulunmaktadır."
    if total_ot > 0 and absences > 0:
        msg = f"Mesai: {total_ot:.2f} saat. UYARI: Devamsızlık kaydı mevcut."

    group['Weekly_Notification_Msg'] = msg
    return group

def create_nlp_text(row):
    """HR One / İK Log sistemi için metin talebi/logu oluşturur."""
    status = row['Work_Status']
    err = row['Error_Type']
    ot = row['Overtime_Hours']

    if status == 'Devamsızlık (M)':
        return "Personel mazeretsiz işe gelmedi, İK süreci başlatılmalı."
    elif status in ['Yıllık İzin', 'Raporlu', 'İdari İzin']:
        return f"{status} formu onaylandı, puantaja işlendi."
    elif err == 'Geç Giriş':
        return f"Puantaj anomali: {row['Actual_Start']} giriş, geç kalma nedeni sorgulanmalı."
    elif err == 'Erken Çıkış':
        return "Puantaj anomali: Erken çıkış tespiti, yönetici onayı bekleniyor."
    elif ot > 0:
        return f"{ot} saatlik fazla çalışma onaylandı, ödeme listesine eklendi."
    elif row['Location'] == 'Ev (Remote)':
        return "Hibrit çalışma kaydı: Evden başarılı bağlantı."
    else:
        return "Rutin çalışma kaydı."

# Uygulama
df = df.groupby(['Employee_ID', 'Week_Number'], group_keys=False).apply(calculate_weekly_comm)
df['HR_Log_Text'] = df.apply(create_nlp_text, axis=1)

print("✅ Haftalık iletişim mesajları ve HR Log metinleri oluşturuldu.")

In [ ]:
# HÜCRE 5: Sonuçların Görüntülenmesi ve Kayıt

print("\n--- Veri Seti Özeti ---")
print(f"Toplam Satır: {len(df)}")
print(f"📅 Tarih Aralığı: {df['Day_Str'].min()} - {df['Day_Str'].max()}")

print("\nEn Sık Görülen Anomali Tipleri:")
print(df['Error_Type'].value_counts())

print("\nÖrnek Veri (İlk 10 Satır):")
display_cols = ['Date', 'Employee_ID', 'Department', 'Work_Status', 'Location', 'Actual_Start', 'Overtime_Hours', 'Error_Type', 'Weekly_Notification_Msg']
print(df[display_cols].head(10))

# CSV Olarak Kaydetme
df.to_csv('hr_punch_data_with_scenarios.csv', index=False)
print("\n📁 Dosya kaydedildi: 'hr_punch_data_with_scenarios.csv'")